In [2]:
import numpy as np


# Generating the A matrix (system interaction matrix)

In [3]:
A=np.array([
    [1,1,0],
    [0,1,1],
    [1,1,1],
    [2,2,0]
], dtype=float)

In [4]:
rank_A=np.linalg.matrix_rank(A)
print("Rank of A:", rank_A)

Rank of A: 3


Hence A is a rank-deficient matrix.

# Three physical quantities observed by four sensors

In [55]:
#Ground truth
x_true=np.array([3,4,3], dtype=float)

In [56]:
#Clean measurements
b_clean=A @ x_true

In [57]:
#Correlated noise covariance
Sigma=np.array([
    [0.2, 0.1, 0.0, 0.05],
    [0.1, 0.2, 0.1, 0.0 ],
    [0.0, 0.1, 0.2, 0.1 ],
    [0.05, 0.0, 0.1, 0.2 ]
])

In [58]:
# Generate correlated noise
noise = np.random.multivariate_normal(
    mean=np.zeros(4),
    cov=Sigma
)


In [59]:
# Noisy measurements
b = b_clean + noise

Thus b is never equal to A @ x_true.

In [60]:
U, S, Vt = np.linalg.svd(A)

tol = 1e-10 #tolerance for singular values
null_space = Vt.T[:, S < tol]
print("Null space of A:", null_space)

Null space of A: []


In [61]:
cond_A = np.linalg.cond(A)
print("Condition number of A:", cond_A)
print("Condition number from singular values:", S[0] / S[-1])


Condition number of A: 7.866945653318726
Condition number from singular values: 7.866945653318725


In [62]:
AtA = A.T @ A

eigvals_AtA, eigvecs_AtA = np.linalg.eig(AtA)

print("Eigenvalues of A^T A:")
print(eigvals_AtA)

print("\nEigenvectors of A^T A:")
print(eigvecs_AtA)


Eigenvalues of A^T A:
[12.94392154  0.20914793  1.84693053]

Eigenvectors of A^T A:
[[-0.65727018 -0.62554549 -0.42034361]
 [-0.72847433  0.67028426  0.14157742]
 [-0.19318659 -0.39926415  0.89625169]]


In [63]:
AAt = A @ A.T

eigvals_AAt, eigvecs_AAt = np.linalg.eig(AAt)

print("Eigenvalues of A A^T:")
print(eigvals_AAt)

print("\nEigenvectors of A A^T:")
print(eigvecs_AAt)


Eigenvalues of A A^T:
[ 1.29439215e+01  1.84693053e+00 -4.57678508e-16  2.09147931e-01]

Eigenvectors of A A^T:
[[-3.85168029e-01 -2.05123216e-01 -8.94427191e-01 -9.78266597e-02]
 [-2.56175880e-01  7.63660919e-01  7.36049830e-16 -5.92617852e-01]
 [-4.38864289e-01  4.54361455e-01 -3.70689790e-16  7.75212103e-01]
 [-7.70336059e-01 -4.10246432e-01  4.47213595e-01 -1.95653319e-01]]


In [64]:
#SVD
AtA = A.T @ A
print(AtA)
eigvals, eigvecs = np.linalg.eig(AtA)

# sort in descending order
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

print("Eigenvalues:", eigvals)
print("Eigenvectors:\n", eigvecs)


[[6. 6. 1.]
 [6. 7. 2.]
 [1. 2. 2.]]
Eigenvalues: [12.94392154  1.84693053  0.20914793]
Eigenvectors:
 [[-0.65727018 -0.42034361 -0.62554549]
 [-0.72847433  0.14157742  0.67028426]
 [-0.19318659  0.89625169 -0.39926415]]


In [65]:
singular_values = np.sqrt(eigvals)
print("Singular values:", singular_values)
V = eigvecs
U = np.zeros((A.shape[0], len(singular_values)))

for i in range(len(singular_values)):
    U[:, i] = (A @ V[:, i]) / singular_values[i]
U = U / np.linalg.norm(U, axis=0)
Sigma = np.diag(singular_values)

for i in range(len(singular_values)):
    Sigma[i, i] = singular_values[i]

print("Sigma:\n", Sigma)
A_reconstructed = U @ Sigma @ V.T
print("Reconstruction error:", np.linalg.norm(A - A_reconstructed))


Singular values: [3.59776619 1.35901822 0.45732694]
Sigma:
 [[3.59776619 0.         0.        ]
 [0.         1.35901822 0.        ]
 [0.         0.         0.45732694]]
Reconstruction error: 2.0539840288898652e-15


In [66]:
b_proj = U @ U.T @ b
residual = b - b_proj
print("A^T residual:", A.T @ residual)


A^T residual: [-1.24344979e-14  2.22044605e-14 -4.44089210e-15]


In [67]:
AtA = A.T @ A
Atb = A.T @ b

x_gauss = np.linalg.solve(AtA, Atb)
print("Gauss solution:", x_gauss)

Gauss solution: [3.10478553 3.68303079 3.28992945]


In [68]:
#Numerical stability check
relative_error=x_true-x_gauss
print("Relative error:", relative_error)

Relative error: [-0.10478553  0.31696921 -0.28992945]


In [69]:
#Custom implementation of Gaussian elimination
def row_reduce(A, b, demo_mode=False):
    """To avoid modifying the matrix and vector specified as input,
    they are copied to new arrays, with the method .copy()
    Warning: it does not work to say "U = A" and "c = b";
    this makes these names synonyms, referring to the same stored data.
    """    
    U = A.copy()
    c = b.copy()
    n = len(b)
    # The function zeros_like() is used to create L with the same size and shape as A, and with all its elements zero initially.
    L = np.zeros_like(A)
    for k in range(n-1):
        if demo_mode: print(f"Step {k=}")
        # compute all the L values for column k:
        L[k+1:,k] = U[k+1:n,k] / U[k,k]  # Beware the case where U[k,k] is 0
        if demo_mode:
            print(f"The multipliers in column {k+1} are {L[k+1:,k]}")
        U[k+1:n,k+1:n] -= L[k+1:n,[k]] @ U[[k],k+1:n]  # The new "outer product" method.
        # Insert the below-diagonal zeros in column k;
        # this is not important for calculations, since those elements of U are not used in backward substitution,
        # but it helps for displaying results and for checking the results via residuals.
        U[k+1:n,k] = 0.0
        
        c[k+1:n] -= L[k+1:n,k] * c[k]  # update c values
        if demo_mode:
            U[k+1:n, k] = 0. # insert zeros in column k of U:
            print(f"The updated matrix is\n{U}")
            print(f"The updated right-hand side is\n{c}")
    return (U, c)

In [70]:
(U, c) = row_reduce(A, b, demo_mode=True)
print()
print(f"U =\n{U}")
print(f"c = {c}")

Step k=0
The multipliers in column 1 are [0. 1. 2.]
The updated matrix is
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]
The updated right-hand side is
[6.72290468 6.97296023 3.35484109 0.16227909]
Step k=1
The multipliers in column 2 are [0. 0.]
The updated matrix is
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]
The updated right-hand side is
[6.72290468 6.97296023 3.35484109 0.16227909]
Step k=2
The multipliers in column 3 are [0.]
The updated matrix is
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]
The updated right-hand side is
[6.72290468 6.97296023 3.35484109 0.16227909]

U =
[[1. 1. 0.]
 [0. 1. 1.]
 [0. 0. 1.]
 [0. 0. 0.]]
c = [6.72290468 6.97296023 3.35484109 0.16227909]


In [71]:
def backward_substitution(U, c, demo_mode=False):
    """Solve U x = c for b."""
    n = len(c)
    x = np.zeros(n)
    x[-1] = c[-1]/U[-1,-1]
    if demo_mode: print(f"x_{n} = {x[-1]}")
    for i in range(2, n):
        x[-i] = (c[-i] - sum(U[-i,1-i:] * x[1-i:])) / U[-i,-i]
        if demo_mode: print(f"x_{n-i+1} = {x[-i]}")
    return x

In [72]:
x = backward_substitution(U, c, demo_mode=True)
print(f"x = {x}")
r = b - A@x
print()
print(f"The residual b - Ax = {r},")
print(f"with maximum norm {max(abs(r)):.3}.")

x_4 = inf
x_3 = -inf
x_2 = nan
x = [  0.  nan -inf  inf]


/tmp/ipython-input-1516058891.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  x[-1] = c[-1]/U[-1,-1]
/tmp/ipython-input-1516058891.py:8: RuntimeWarning: invalid value encountered in scalar add
  x[-i] = (c[-i] - sum(U[-i,1-i:] * x[1-i:])) / U[-i,-i]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)

Hence, as this is an ill-conditioned matrix, our custom method is not working well.

In [ ]:
#sensitivity to noise
noise_level = 1e-4
A_noisy = A + np.random.normal(0, noise_level, A.shape)
b_noisy = b + np.random.normal(0, noise_level, b.shape)
x_noisy = np.linalg.solve(A_noisy, b_noisy)
error = np.linalg.norm(x_noisy - x_gauss) / np.linalg.norm(x_gauss)
print(f"Sensitivity (Relative Error): {error:.2e}")


LinAlgError: Last 2 dimensions of the array must be square

Therefore, we can see that as proof of an ill-conditioned matrix, the Gaussian elimination method is not working for noisy matrices, and the relative error is very high.


In [ ]:
from scipy.linalg import lu_factor, lu_solve

lu, piv = lu_factor(AtA)
x_lu = lu_solve((lu, piv), Atb)
print("LU solution:", x_lu)

LU solution: [3.57026015 3.46413555 3.51285164]


In [ ]:
Q, R = np.linalg.qr(A, mode='reduced')
x_qr = np.linalg.solve(R, Q.T @ b)
print("QR solution:", x_qr)

QR solution: [3.57026015 3.46413555 3.51285164]


In [ ]:
x_pinv = np.linalg.pinv(A) @ b
print("Pseudoinverse solution:", x_pinv)

Pseudoinverse solution: [3.57026015 3.46413555 3.51285164]


In [ ]:
def jacobi(A, b, x0, max_iter=1000, tol=1e-6):
    D = np.diag(np.diag(A))
    R = A - D
    x = x0.copy()
    for _ in range(max_iter):
        x_new = np.linalg.inv(D) @ (b - R @ x)
        if np.linalg.norm(x_new - x) < tol:
            break
        x = x_new
    return x

x_jacobi = jacobi(AtA, Atb, np.zeros(3))
print("Jacobi solution:", x_jacobi)

Jacobi solution: [-1.17321590e+83 -1.17588690e+83 -1.45742825e+83]


In [ ]:
def gauss_seidel(A, b, x0, max_iter=1000, tol=1e-6):
    x = x0.copy()
    for _ in range(max_iter):
        x_old = x.copy()
        for i in range(len(b)):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i+1:], x_old[i+1:])
            x[i] = (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x - x_old) < tol:
            break
    return x

x_gs = gauss_seidel(AtA, Atb, np.zeros(3))
print("Gauss-Seidel solution:", x_gs)

Gauss-Seidel solution: [3.57026829 3.46412728 3.51285584]


In [ ]:
noise = 0.01 * np.random.randn(4)
b_noisy = b + noise
x_qr_noisy   = np.linalg.solve(R, Q.T @ b_noisy)
x_pinv_noisy = np.linalg.pinv(A) @ b_noisy
print("QR solution with noise:", x_qr_noisy)
print("Pseudoinverse solution with noise:", x_pinv_noisy)

QR solution with noise: [3.56690654 3.4698175  3.51011766]
Pseudoinverse solution with noise: [3.56690654 3.4698175  3.51011766]
